## Project Description and Goal

<font color="blue">
We've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books.<br> We have to retrive some data from the tables. This information will be used to generate a value proposition for a new product.

## Open the data file and study the general information

In [3]:
all_data=f'''SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(all_data, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
all_data=f'''SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(all_data, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
all_data=f'''SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(all_data, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
all_data=f'''SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(all_data, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
all_data=f'''SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(all_data, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Make SQL queries for the tasks

 - **Find the number of books released after January 1, 2000.**

In [8]:
data=f'''
SELECT 
  COUNT(book_id) AS book_num
FROM 
  books
WHERE publication_date >'2000-01-01'
'''
pd.io.sql.read_sql(data, con = engine)

,book_num
0,819


<font color="blue">
There ere 819 books released after 2000-01-01. 

 - **Find the number of user reviews and the average rating for each book.**

In [9]:
data=f'''
SELECT 
  books.book_id,
  books.title,
  subq1.avg_rate,
  subq2.num_review
  
FROM books

INNER JOIN 
 (SELECT 
    book_id, 
    AVG(rating) As avg_rate
  FROM ratings
  GROUP BY book_id
  ORDER BY book_id) AS subq1
ON  books.book_id= subq1.book_id

INNER JOIN 
 (SELECT 
    book_id, 
    COUNT(text) As num_review
  FROM reviews
  GROUP BY book_id
  ORDER BY book_id) AS subq2
  
ON books.book_id=subq2.book_id
ORDER BY num_review DESC
'''    
pd.io.sql.read_sql(data, con = engine)

,book_id,title,avg_rate,num_review
0,948,Twilight (Twilight #1),3.662500,7
1,854,The Road,3.772727,6
2,656,The Book Thief,4.264151,6
3,734,The Glass Castle,4.206897,6
4,963,Water for Elephants,3.977273,6
...,...,...,...,...
989,465,Naked Empire (Sword of Truth #8),3.500000,1
990,446,Moo Baa La La La!,3.000000,1
991,431,Merrick (The Vampire Chronicles #7),4.000000,1
992,92,Babyville,3.500000,1


<font color="blue">
    
We found the number of user reviews and the average rating for each book.<br> The most popular books are on average rated with 5.   <br>
The most reviews are related to the book `Twilight`.

- **Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).**

In [10]:
data=f'''
SELECT
 subq.publisher_id,
 publishers.publisher,
 subq.book_num
 
FROM publishers
INNER JOIN
(SELECT 
  publisher_id,
  COUNT(book_id) AS book_num
  
FROM 
  books
WHERE num_pages >=50 
GROUP BY publisher_id) AS subq

ON publishers.publisher_id=subq.publisher_id
ORDER BY book_num DESC
LIMIT 1
'''
pd.io.sql.read_sql(data, con = engine)

,publisher_id,publisher,book_num
0,212,Penguin Books,42


<font color="blue">
     
`Penguin Books` is the publisher, that has released the greatest number of books(amoung books with more than 50 pages).


- **Identify the author with the highest average book rating: look only at books with at least 50 ratings.**

In [11]:
data=f'''
SELECT 
 
  authors.author,
  AVG(subq.avg_rate) AS avg_rate
  
FROM
 authors
INNER JOIN 

(SELECT 
  books.author_id,
  subq1.rate_num,
  subq2.avg_rate
  
FROM books
INNER JOIN 
 (SELECT 
    book_id,
    COUNT(rating) As rate_num
    
  FROM ratings
  GROUP BY book_id) AS subq1
ON  books.book_id= subq1.book_id

INNER JOIN 
 (SELECT 
    book_id, 
    AVG(rating) As avg_rate
    
  FROM ratings
  GROUP BY book_id) AS subq2
ON  books.book_id= subq2.book_id) AS subq
ON subq.author_id=authors.author_id

WHERE subq.rate_num>=50 

GROUP BY author
ORDER BY avg_rate DESC
LIMIT 1
'''    
pd.io.sql.read_sql(data, con = engine)

,author,avg_rate
0,J.K. Rowling/Mary GrandPré,4.283844


<font color="blue">
    
`J.K. Rowling/Mary GrandPré` is The most popular author with the highest average book rating (amoung books with at least 50 ratings) 
    


- **Find the average number of text reviews among users who rated more than 50 books.**

In [12]:
data=f'''
SELECT 
  
  AVG(subq2.num_text)AS avg_num_text
  
FROM
  (SELECT 
    username,
    COUNT(book_id) As num_rate_books
    
  FROM ratings
  GROUP BY username) As subq1

LEFT JOIN 
  (SELECT 
    username,
    COUNT(text) As num_text
    
  FROM reviews
  GROUP BY username) As subq2
ON  subq1.username= subq2.username

WHERE subq1.num_rate_books>50 

'''    
pd.io.sql.read_sql(data, con = engine)

,avg_num_text
0,24.333333


<font color="blue">
users who rated more than 50 books, made also on average 24 text reviews.    